In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d prashant268/chest-xray-covid19-pneumonia

Saving kaggle.json to kaggle.json
 99% 2.03G/2.06G [00:46<00:00, 26.6MB/s]
100% 2.06G/2.06G [00:46<00:00, 47.0MB/s]


In [ ]:
!unzip chest-xray-covid19-pneumonia.zip

Streaming output truncated to the last 5000 lines.
  inflating: Data/train/COVID19/COVID19(229).jpg  
  inflating: Data/train/COVID19/COVID19(23).jpg  
  inflating: Data/train/COVID19/COVID19(230).jpg  
  inflating: Data/train/COVID19/COVID19(231).jpg  
  inflating: Data/train/COVID19/COVID19(232).jpg  
  inflating: Data/train/COVID19/COVID19(233).jpg  
  inflating: Data/train/COVID19/COVID19(234).jpg  
  inflating: Data/train/COVID19/COVID19(235).jpg  
  inflating: Data/train/COVID19/COVID19(236).jpg  
  inflating: Data/train/COVID19/COVID19(237).jpg  
  inflating: Data/train/COVID19/COVID19(238).jpg  
  inflating: Data/train/COVID19/COVID19(239).jpg  
  inflating: Data/train/COVID19/COVID19(24).jpg  
  inflating: Data/train/COVID19/COVID19(240).jpg  
  inflating: Data/train/COVID19/COVID19(241).jpg  
  inflating: Data/train/COVID19/COVID19(242).jpg  
  inflating: Data/train/COVID19/COVID19(243).jpg  
  inflating: Data/train/COVID19/COVID19(244).jpg  
  inflating: Data/train/COVID19/C

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten,Conv2D,MaxPooling2D,Dropout,Flatten
from keras.models import Model,Sequential
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
def preprocess_input(x):
  img=keras.applications.xception.preprocess_input(img_to_array(x))
  return array_to_img(x)

IMAGE_SIZE = [224, 224]


train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

training_set = train_datagen.flow_from_directory('/content/Data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/Data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import Xception
model_3=Xception(include_top=False,weights="imagenet",input_shape=(224, 224, 3),pooling='max',classes=1000)
folders = glob('/content/Data/train/*')

x = Flatten()(model_3.output)
prediction = Dense(len(folders), activation='softmax')(x)

Yashvi = Model(inputs=model_3.input, outputs=prediction)
Yashvi.summary()

83689472/83683744 [==============================] - 1s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________

In [ ]:
Yashvi.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']) 
Yashvi.fit_generator(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
161/161 [==============================] - 231s 1s/step - loss: 0.2394 - accuracy: 0.9149 - val_loss: 1.0267 - val_accuracy: 0.7500
Epoch 2/10
161/161 [==============================] - 227s 1s/step - loss: 0.1327 - accuracy: 0.9555 - val_loss: 0.7642 - val_accuracy: 0.7686
Epoch 3/10
161/161 [==============================] - 224s 1s/step - loss: 0.1237 - accuracy: 0.9582 - val_loss: 0.3222 - val_accuracy: 0.9130
Epoch 4/10
161/161 [==============================] - 223s 1s/step - loss: 0.0972 - accuracy: 0.9670 - val_loss: 0.1551 - val_accuracy: 0.9402
Epoch 5/10
161/161 [==============================] - 223s 1s/step - loss: 0.0778 - accuracy: 0.9716 - val_loss: 0.2045 - val_accuracy: 0.9317
Epoch 6/10
161/161 [==============================] - 224s 1s/step - loss: 0.0891 - accuracy: 0.9673 - val_loss: 0.0899 - val_accuracy: 0.9689
Epoch 7/10
161/161 [==============================] - 222s 1s/step

In [ ]:
Yashvi.save('covid19.h5')